## Importamos librerias

In [31]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from lightgbm import early_stopping
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from datetime import timedelta
from datetime import datetime

In [30]:
# Cargar el dataset con las entregas reales
df = pl.read_csv("results.csv") ## CHEQUEAR  ACA

# Convertir la columna de fechas a formato de fecha
df = df.with_columns(pl.col('date').str.strptime(pl.Date, '%Y-%m-%d'))

# Mostrar las primeras filas para verificar la carga
print(df)
print(df['date'].max())

shape: (1_746, 4)
┌────────────┬────────┬────────────┬────────────────┐
│ date       ┆ true   ┆ pred       ┆ pred_corrected │
│ ---        ┆ ---    ┆ ---        ┆ ---            │
│ date       ┆ f64    ┆ f64        ┆ f64            │
╞════════════╪════════╪════════════╪════════════════╡
│ 2019-10-09 ┆ 4374.0 ┆ 4311.143   ┆ 4311.143       │
│ 2019-10-10 ┆ 4197.0 ┆ 4251.694   ┆ 4251.694       │
│ 2019-10-11 ┆ 3460.0 ┆ 3763.6528  ┆ 3763.6528      │
│ 2019-10-12 ┆ 1184.0 ┆ 1302.8416  ┆ 1302.8416      │
│ 2019-10-13 ┆ 444.0  ┆ -121.78247 ┆ -121.78247     │
│ …          ┆ …      ┆ …          ┆ …              │
│ 2024-08-04 ┆ 709.0  ┆ -327.0537  ┆ -150.48712     │
│ 2024-08-05 ┆ 6655.0 ┆ 5325.941   ┆ 5540.0024      │
│ 2024-08-06 ┆ 6851.0 ┆ 5941.698   ┆ 6358.4595      │
│ 2024-08-07 ┆ 6404.0 ┆ 5289.981   ┆ 5603.1885      │
│ 2024-08-08 ┆ 3669.0 ┆ 5761.5947  ┆ 6270.6396      │
└────────────┴────────┴────────────┴────────────────┘
2024-08-08


## Calcular el error Ei = Ri - Pi

In [3]:
df = df.with_columns(
    (pl.col("true") - pl.col("pred")).alias("error")
)
print(df)

shape: (1_746, 5)
┌────────────┬────────┬────────────┬────────────────┬────────────┐
│ date       ┆ true   ┆ pred       ┆ pred_corrected ┆ error      │
│ ---        ┆ ---    ┆ ---        ┆ ---            ┆ ---        │
│ date       ┆ f64    ┆ f64        ┆ f64            ┆ f64        │
╞════════════╪════════╪════════════╪════════════════╪════════════╡
│ 2019-10-09 ┆ 4374.0 ┆ 4311.143   ┆ 4311.143       ┆ 62.857     │
│ 2019-10-10 ┆ 4197.0 ┆ 4251.694   ┆ 4251.694       ┆ -54.694    │
│ 2019-10-11 ┆ 3460.0 ┆ 3763.6528  ┆ 3763.6528      ┆ -303.6528  │
│ 2019-10-12 ┆ 1184.0 ┆ 1302.8416  ┆ 1302.8416      ┆ -118.8416  │
│ 2019-10-13 ┆ 444.0  ┆ -121.78247 ┆ -121.78247     ┆ 565.78247  │
│ …          ┆ …      ┆ …          ┆ …              ┆ …          │
│ 2024-08-04 ┆ 709.0  ┆ -327.0537  ┆ -150.48712     ┆ 1036.0537  │
│ 2024-08-05 ┆ 6655.0 ┆ 5325.941   ┆ 5540.0024      ┆ 1329.059   │
│ 2024-08-06 ┆ 6851.0 ┆ 5941.698   ┆ 6358.4595      ┆ 909.302    │
│ 2024-08-07 ┆ 6404.0 ┆ 5289.981   ┆ 5603.18

### Tipo de feriado

In [4]:
# Cargar el dataset con los tipos de feriados
df_feriado = pl.read_csv("tipo_feriados.csv")
df_feriado = df_feriado.with_columns(pl.col('date').str.strptime(pl.Date, '%Y-%m-%d'))
print(df_feriado)

shape: (1_812, 6)
┌────────────┬───────────────────┬────────────┬──────────────┬─────────┬──────────────┐
│ date       ┆ cantidad_entregas ┆ is_holiday ┆ holiday_name ┆ weekday ┆ tipo_feriado │
│ ---        ┆ ---               ┆ ---        ┆ ---          ┆ ---     ┆ ---          │
│ date       ┆ i64               ┆ bool       ┆ str          ┆ i64     ┆ i64          │
╞════════════╪═══════════════════╪════════════╪══════════════╪═════════╪══════════════╡
│ 2019-09-05 ┆ 2769              ┆ false      ┆ null         ┆ 4       ┆ 0            │
│ 2019-09-06 ┆ 8474              ┆ false      ┆ null         ┆ 5       ┆ 0            │
│ 2019-09-07 ┆ 613               ┆ false      ┆ null         ┆ 6       ┆ -1           │
│ 2019-09-08 ┆ 346               ┆ false      ┆ null         ┆ 7       ┆ -1           │
│ 2019-09-09 ┆ 4814              ┆ false      ┆ null         ┆ 1       ┆ 0            │
│ …          ┆ …                 ┆ …          ┆ …            ┆ …       ┆ …            │
│ 2024-08-04 ┆

In [5]:
df_feriado = df_feriado.select(['date', 'holiday_name', 'tipo_feriado'])
df = df.join(df_feriado, on='date', how='left')
print(df)

shape: (1_758, 7)
┌────────────┬────────┬────────────┬────────────────┬────────────┬──────────────────┬──────────────┐
│ date       ┆ true   ┆ pred       ┆ pred_corrected ┆ error      ┆ holiday_name     ┆ tipo_feriado │
│ ---        ┆ ---    ┆ ---        ┆ ---            ┆ ---        ┆ ---              ┆ ---          │
│ date       ┆ f64    ┆ f64        ┆ f64            ┆ f64        ┆ str              ┆ i64          │
╞════════════╪════════╪════════════╪════════════════╪════════════╪══════════════════╪══════════════╡
│ 2019-10-09 ┆ 4374.0 ┆ 4311.143   ┆ 4311.143       ┆ 62.857     ┆ Yom Kippur       ┆ 2            │
│ 2019-10-10 ┆ 4197.0 ┆ 4251.694   ┆ 4251.694       ┆ -54.694    ┆ null             ┆ 0            │
│ 2019-10-11 ┆ 3460.0 ┆ 3763.6528  ┆ 3763.6528      ┆ -303.6528  ┆ null             ┆ 0            │
│ 2019-10-12 ┆ 1184.0 ┆ 1302.8416  ┆ 1302.8416      ┆ -118.8416  ┆ Day of Respect   ┆ 2            │
│            ┆        ┆            ┆                ┆            ┆ for Cu

### Variación del feriado

In [6]:
# Cargar el dataset con la variacion de cada feriado
df_variacion = pl.read_parquet("variacion_tipo_feriados.parquet")
print(df_variacion)

shape: (44, 5)
┌───────────────┬───────────────────────────┬──────────────┬──────────────────┬────────────────────┐
│ grupo_feriado ┆ holiday_name              ┆ tipo_feriado ┆ dates            ┆ promedio_variacion │
│ ---           ┆ ---                       ┆ ---          ┆ ---              ┆ ---                │
│ i64           ┆ str                       ┆ i64          ┆ list[list[date]] ┆ f64                │
╞═══════════════╪═══════════════════════════╪══════════════╪══════════════════╪════════════════════╡
│ 1             ┆ Sixth Day of Passover     ┆ 2            ┆ [[2020-04-14],   ┆ 15.331127          │
│               ┆                           ┆              ┆ [2021-04-02], …… ┆                    │
│ 2             ┆ Rosh Hashana Eve          ┆ 2            ┆ [[2019-09-29],   ┆ 3.804329           │
│               ┆                           ┆              ┆ [2020-09-18], …… ┆                    │
│ 3             ┆ Second Day of Rosh        ┆ 2            ┆ [[2019-10-01], 

In [7]:
# Asignar 1 como valor por defecto para la columna `promedio_variacion`
df = df.with_columns(pl.lit(1).alias('promedio_variacion'))

# Iterar sobre cada fila en el DataFrame con las variaciones
for row in df_variacion.iter_rows(named=True):
    variacion = row['promedio_variacion']
    fechas = row['dates']  # Es una lista de listas de fechas

    # Iterar sobre las listas de fechas dentro de 'dates'
    for lista_fechas in fechas:
        for fecha in lista_fechas:
            # Asignar la variación a la fecha correspondiente en df_original
            df = df.with_columns(
                pl.when(pl.col('date') == fecha).then(variacion).otherwise(pl.col('promedio_variacion')).alias('promedio_variacion')
            )

# Revisa el DataFrame final
print(df)


shape: (1_758, 8)
┌────────────┬────────┬────────────┬────────────┬────────────┬────────────┬────────────┬───────────┐
│ date       ┆ true   ┆ pred       ┆ pred_corre ┆ error      ┆ holiday_na ┆ tipo_feria ┆ promedio_ │
│ ---        ┆ ---    ┆ ---        ┆ cted       ┆ ---        ┆ me         ┆ do         ┆ variacion │
│ date       ┆ f64    ┆ f64        ┆ ---        ┆ f64        ┆ ---        ┆ ---        ┆ ---       │
│            ┆        ┆            ┆ f64        ┆            ┆ str        ┆ i64        ┆ f64       │
╞════════════╪════════╪════════════╪════════════╪════════════╪════════════╪════════════╪═══════════╡
│ 2019-10-09 ┆ 4374.0 ┆ 4311.143   ┆ 4311.143   ┆ 62.857     ┆ Yom Kippur ┆ 2          ┆ -8.250973 │
│ 2019-10-10 ┆ 4197.0 ┆ 4251.694   ┆ 4251.694   ┆ -54.694    ┆ null       ┆ 0          ┆ 1.0       │
│ 2019-10-11 ┆ 3460.0 ┆ 3763.6528  ┆ 3763.6528  ┆ -303.6528  ┆ null       ┆ 0          ┆ 1.0       │
│ 2019-10-12 ┆ 1184.0 ┆ 1302.8416  ┆ 1302.8416  ┆ -118.8416  ┆ Day of    

## Predicción de errores

### LightGBM

Basado en el algoritmo de boosting de árboles de decisión.

Primero dividimos los datos

In [25]:
X = df[['pred', 'tipo_feriado', 'promedio_variacion']]  
y = df['error'] 

X = X.to_numpy()
y = y.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Entrenamiento del modelo

In [32]:
# Crear el dataset de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

Definimos params de busqueda

In [39]:
# Espacio de hiperparámetros a explorar
param_dist = {
    'num_leaves': np.arange(20, 150, 10),  # número de hojas en cada árbol
    'learning_rate': np.linspace(0.01, 0.1, 10),  # tasa de aprendizaje
    'n_estimators': np.arange(100, 1000, 100),  # número de árboles
    'max_depth': np.arange(3, 15, 1),  # profundidad máxima del árbol
   # 'min_child_samples': np.arange(5, 100, 5),  # número mínimo de muestras en un nodo hoja
   # 'subsample': np.linspace(0.5, 1.0, 10),  # fracción de datos utilizada para construir cada árbol
   # 'colsample_bytree': np.linspace(0.5, 1.0, 10)  # fracción de columnas usada para construir cada árbol
}

In [40]:
lgb_model = lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', verbose=-1)

In [41]:
random_search = RandomizedSearchCV(estimator=lgb_model, 
                                   param_distributions=param_dist, 
                                   n_iter=10,  # Número de combinaciones a probar
                                   scoring='neg_root_mean_squared_error', 
                                   cv=5,  # Validación cruzada con 5 folds
                                   verbose=1, 
                                   random_state=42, 
                                   n_jobs=-1)  # Utilizar todos los núcleos disponibles

In [42]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=LGBMRegressor(metric='rmse',
                                           objective='regression', verbose=-1),
                   n_jobs=-1,
                   param_distributions={'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ]),
                                        'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'n_estimators': array([100, 200, 300, 400, 500, 600, 700, 800, 900]),
                                        'num_leaves': array([ 20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130, 140])},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [43]:
# Obtener los mejores parámetros
best_params = random_search.best_params_
print(f"Mejores parámetros encontrados: {best_params}")

Mejores parámetros encontrados: {'num_leaves': 30, 'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.05000000000000001}


In [44]:
# Hacer predicciones
y_pred = random_search.predict(X_test)
print(y_pred)

[   99.42191293   389.04297212  -105.87791827   633.65603304
   205.87368202   -92.7117026    -61.12578952   226.12035006
   226.12035006   226.12035006   228.66032686  -105.87791827
   213.9340322    228.66032686   333.22758338   121.61753803
   343.76695636   254.27665092   -61.12578952  -281.74332823
   295.74756973   222.89846119   254.06454108   254.27665092
  -105.87791827   345.78913871   490.89594828   303.90038959
  -427.04231147  -102.61560095   299.66499784   174.58387603
  -193.65823442  -160.8259689    -53.15933122  -155.48678727
   295.74756973  -193.65823442   490.89594828   285.3987568
   254.27665092   251.37910636   295.74756973  -509.55857728
  -155.48678727   205.87368202   490.89594828   -14.45994346
   251.37910636   205.87368202  -155.48678727   232.36160061
 -1178.90766805   213.9340322    427.13830532   213.9340322
   299.66499784   333.52120461   490.89594828   232.36160061
   254.06454108   345.78913871   633.65603304   343.76695636
   345.78913871   431.6151

In [48]:
print(y_test)

[  -54.0979     1394.782      -144.3601     1844.3152     -747.1426
   290.377      -320.09033     875.1357      394.3745     -107.2322
   311.432       -61.96387     479.3643     -308.0034     1346.916
    54.749       319.3516     3579.9604      -78.55273     207.42
    83.834       140.6924     -795.729      -151.7266     -134.4568
   889.307      1222.7517      378.23535    -443.257      -686.2607
  -682.7915      735.434       -75.7405      801.8274       56.4458
   380.9628     -359.019      -772.5962      292.8638      226.0967
   -23.8384     -232.7593      525.6104    -1061.4326    -1141.7528
   258.0293      276.7188      356.324       201.152       300.9414
  -371.6118      338.66553     264.4978      467.9634      -81.6885
  -163.176      1380.323       500.712      -216.5737      131.94263
   -50.9683      264.36        915.524       130.562      -167.4834
   -92.755       622.227       372.417       -98.063       362.6426
   776.519      -843.018         6.2532     1229.2

### Evaluación del modelo

In [46]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE en el set de prueba: {rmse}")

RMSE en el set de prueba: 697.1637669794615


#### MAPE

In [49]:
def calculate_mape(y_true, y_pred):
    # Evitar divisiones por cero
    nonzero_indices = y_true != 0
    y_true_nonzero = y_true[nonzero_indices]
    y_pred_nonzero = y_pred[nonzero_indices]
    
    mape = np.mean(np.abs((y_true_nonzero - y_pred_nonzero) / y_true_nonzero)) * 100
    return mape

In [50]:
mape = calculate_mape(y_test, y_pred)
print(f"MAPE: {mape:.2f}%")

MAPE: 158.51%
